In [ ]:
#Cài đặt thư viện cần thiết
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import random
import re
import time
from datetime import datetime

#Mở trang web để cào dữ liệu

options = webdriver.ChromeOptions()
options.add_argument("--lang=en-US") #chọn ngôn ngữ web là tiếng Anh
options.add_argument("--disable-gpu") #thay vì sử dụng đơn vị xử lý đồ họa (GPU) của máy tính để hiển thị đồ họa mà sẽ chỉ dựa vào đơn vị xử lý trung tâm (CPU).
options.add_argument("--no-sandbox") #không sử dụng chế độ sandbox
options.add_argument("--disable-dev-shm-usage") #không sử dụng bộ nhớ chia sẻ mà chỉ sử dụng đĩa để lưu trữ tạm thời
options.add_argument("--disable-images") #vô hiệu hóa tải hình ảnh
options.add_argument("--blink-settings=imagesEnabled=false")  # Vô hiệu hóa hình ảnh
options.add_argument("--disable-extensions")  # vô hiệu hóa các tiện ích mở rộng
options.add_argument("--disable-popup-blocking")  #vô hiệu hóa chặn popup
options.add_argument("--disable-infobars")  # vô hiệu hóa các thông báo
options.add_argument("--mute-audio")  # vô hiệu hóa âm thanh

# Tạo phiên bản WebDriver
driver = webdriver.Chrome(options=options)

url = 'https://www.youtube.com/playlist?list=PLLsO3pH5RkATp-GHK309rE9_JfIxeTSgm'

driver.get(url)


In [ ]:
#Hàm cào dữ liệu về tiêu đề, lượt xem, ngày đăng, lượt thích của video
def crawl_data(titles,views,published_date,likes):
    try:
        wait = WebDriverWait(driver, 15)
        #Cào tiêu đề
        title_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.ytd-watch-metadata')))
        title = title_element.text

        #Cào lượt thích
        like_button = driver.find_element(By.XPATH,  '//*[@id="top-level-buttons-computed"]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button')
        like_count = like_button.get_attribute("aria-label")
        like_count = str(re.findall(r'\b\d+\b', like_count)[0])

        #Ấn vào mục "more..." để cào rõ ngày đăng và lượt xem
        driver.find_element(By.CSS_SELECTOR, '#description-inline-expander #expand').click()
        time.sleep(3)

        #Cào lượt xem
        view = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]').text

        #Cào ngày đăng
        date = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]').text

        #Thêm vào danh sách
        titles.append(title)
        views.append(view)
        published_date.append(date)
        likes.append(like_count)

    except Exception as e:
        #Nếu không tìm ra thì lưu vào danh sách là "not found"
        print(f"Error finding title: {e}")
        titles.append("Title not found!")
        views.append("View not found!")
        published_date.append("Date not found!")
        likes.append("Like count not found!")

In [ ]:
def find_ID(starting_ID):
    #Vị trí bắt đầu chạy vòng lặp
    k = 0
    while True:
        #Danh sách chứa vị trí (index) của video trong danh sách phát
        list_of_ids = driver.find_elements(By.ID, 'index')

        #Nếu index cuối cùng trong 1 trang trùng với index được chỉ định
        if int(list_of_ids[-1].text) == starting_ID:
            return list_of_ids[-1]
        #Nếu index cuối cùng trong 1 trang lớn hơn index được chỉ định thì mới duyệt qua từng index trong trang đấy để tìm ra video có index thích hợp
        if int(list_of_ids[-1].text) > starting_ID:
            for num_id in list_of_ids[k:]:
                if num_id.text == str(starting_ID):
                    driver.execute_script("arguments[0].scrollIntoView();", num_id) #tìm được thì kéo thanh cuộn tới video ở index đó
                    time.sleep(2)
                    return num_id

        #Cuộn xuống để tải thêm nội dung
        k = len(list_of_ids) - 1
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
    return None

def get_video_link_id(num_id):
    # Xây dựng XPath mới sử dụng row và col
    xpath = f"/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-playlist-video-list-renderer/div[3]/ytd-playlist-video-renderer[{int(num_id.text) + 1}]/div[2]/div[1]/div/h3/a"
    video_element = num_id.find_element(By.XPATH, xpath)

    return video_element

#Crawl video thứ 100 thì starting_id sẽ là 99. Do mỗi video trong web youtube là 1 index bắt đầu từ 0
starting_id = 216
ID_found = find_ID(starting_id)
if ID_found:
    start_video_link = get_video_link_id(ID_found)
    if start_video_link:
        start_video_url = start_video_link.get_attribute('href') #tìm link video
    else:
        print("Could not find the video link after the target date.")
        driver.quit()
        exit()

else:
    print(f"Target date '{starting_id}' not found.")
    driver.quit()
    exit()

In [ ]:
#Danh sách lưu dữ liệu video
titles = []
views = []
published_date = []
likes = []
collected_links = set()

In [ ]:
while True:
    try:
        video_links = driver.find_elements(By.ID, 'video-title') #tìm link của các video
        start_collecting = False
        end_collecting = False

        for link in video_links:
            video_url = link.get_attribute('href')

            if video_url == start_video_url: #nếu như link này là trùng khớp với link của video có index = starting_ID
                start_collecting = True #bắt đầu cào dữ liệu

            if start_collecting == True:
                if video_url in collected_links:  #nếu đã cào video này thì tiếp tục tìm kiếm
                    continue

                collected_links.add(video_url) #add link của video đã cào vào 1 mảng để lưu trữ

                ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform() #mở link trong tab mới
                driver.switch_to.window(driver.window_handles[-1]) #switch sang tab mới mở
                time.sleep(random.randint(2, 4)) #random thời gian ngủ để tránh bị phát hiện là bot
                try:
                    crawl_data(titles, views, published_date, likes) #tiến hành cào các dữ liệu cần thiết
                finally:
                    driver.close()  #đóng tab sau khi crawl xong 1 video
                    driver.switch_to.window(driver.window_handles[0]) #chuyển về tab cũ (tab có playlist)
                    time.sleep(2)

        if end_collecting==True:  #check điều kiện tiếp tục crawl hay không
            break

        actions = ActionChains(driver)
        actions.send_keys(Keys.END).perform() #cuộn xuống để load thêm video
        time.sleep(random.randint(2, 4))
        new_video_links = driver.find_elements(By.ID, 'video-title')
        if len(new_video_links) == len(video_links): #kiểm tra điều kiện đã tải hết playlist hay chưa
            print("No more new videos to load.")
            break


    except Exception as e:
        print("Error occurred:", e)
        break


driver.quit()

In [ ]:
#tạo dataframe để lưu trữ những thông tin vừa cào được
import pandas as pd
df = pd.DataFrame()
df['title']=titles
df['views']=views
df['published_date']=published_date
df['likes']=likes

In [ ]:
df

,title,views,published_date,likes
0,Nữ bệnh nhân khỏi ung thư nhờ phương pháp điều...,"89,975 views","Premiered Sep 29, 2023",835
1,"“Xẻ thịt” đồi đất ven hồ sông Dinh, khai thác ...","88,128 views","Premiered Feb 26, 2024",500


In [ ]:
#lưu file về máy
df.to_csv('video_data_file1.csv', index=False)